In [1]:
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Join").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/01 09:02:31 WARN Utils: Your hostname, keen-alise-NH5x-NH7x-HHx-HJx-HKx, resolves to a loopback address: 127.0.1.1; using 10.1.11.194 instead (on interface wlp0s20f3)
25/12/01 09:02:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/01 09:02:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
flights = spark.read.csv("nyc_flights.csv", header = True, inferSchema = True)
airlines = spark.read.csv("nyc_airlines.csv", header = True, inferSchema = True)
airports = spark.read.csv("nyc_airports.csv", header = True, inferSchema = True)

In [15]:
flights.createOrReplaceTempView("flights_table")
airlines.createOrReplaceTempView("airlines_table")
airports.createOrReplaceTempView("airports_table")

In [16]:
query1 = "SELECT * FROM flights_table"
result = spark.sql(query1)
result.show()

+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|
+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|2013|    1|  1|     517|           515|        2|     830|           819|       11|     UA|  1545| N14228|   EWR| IAH|     227|    1400|   5|    15|2013-01-01 15:45:00|
|2013|    1|  1|     533|           529|        4|     850|           830|       20|     UA|  1714| N24211|   LGA| IAH|     227|    1416|   5|    29|2013-01-01 15:45:00|
|2013|    1|  1|     542|           540|        2|     923|           850|       33|     AA|  1141| N619AA|   JFK| MIA|     160|    1089|   5|    40|2

In [17]:
''' Which airline had the highest no. of delayed departures? ''' 
query1 = """
SELECT a.name, COUNT(*) AS delayed_departures
FROM flights_table f
JOIN airlines_table a ON f.carrier = a.carrier
WHERE TRY_CAST(f.dep_delay AS INT) > 0
GROUP BY a.name
ORDER BY delayed_departures DESC
LIMIT 1
"""
spark.sql(query1).show()

+--------------------+------------------+
|                name|delayed_departures|
+--------------------+------------------+
|United Air Lines ...|             27261|
+--------------------+------------------+



In [18]:
''' Challange2: Which airport has flights arriving most early on average '''
query2 = """
SELECT a.name, AVG(TRY_CAST(f.arr_delay AS INT)) AS avg_arr_delay
FROM flights_table f
JOIN airports_table a ON f.dest = a.faa
WHERE TRY_CAST(f.arr_delay AS INT) <= 0
GROUP BY a.name
ORDER BY avg_arr_delay ASC
LIMIT 1
"""
spark.sql(query2).show()

+--------------------+-------------------+
|                name|      avg_arr_delay|
+--------------------+-------------------+
|Ted Stevens Ancho...|-27.333333333333332|
+--------------------+-------------------+



In [19]:
''' Exercise: Find out the avg, min, and max air time when travelling from JFK to SEA airports. '''

query3 = """
SELECT 
    AVG(TRY_CAST(air_time AS INT)) AS avg_air_time,
    MIN(TRY_CAST(air_time AS INT)) AS min_air_time,
    MAX(TRY_CAST(air_time AS INT)) AS max_air_time
FROM flights_table
WHERE origin = 'JFK' AND dest = 'SEA'
"""
spark.sql(query3).show()

+-----------------+------------+------------+
|     avg_air_time|min_air_time|max_air_time|
+-----------------+------------+------------+
|329.3744578313253|         275|         389|
+-----------------+------------+------------+



In [20]:
''' Find the airline with the worst average arrival delay '''
query4 = """
SELECT a.name, AVG(TRY_CAST(f.arr_delay AS INT)) AS avg_arr_delay
FROM flights_table f
JOIN airlines_table a ON f.carrier = a.carrier
GROUP BY a.name
ORDER BY avg_arr_delay DESC
LIMIT 1
"""

spark.sql(query4).show()

+--------------------+------------------+
|                name|     avg_arr_delay|
+--------------------+------------------+
|Frontier Airlines...|21.920704845814978|
+--------------------+------------------+



In [21]:
''' Which airline is most punctual (on-time %)'''
query_on_time = """
SELECT 
    a.name,
    SUM(CASE 
            WHEN TRY_CAST(f.arr_delay AS INT) <= 0 THEN 1 
            ELSE 0
        END) * 100.0 / COUNT(*) AS on_time_percentage
FROM flights_table f
JOIN airlines_table a ON f.carrier = a.carrier
GROUP BY a.name
ORDER BY on_time_percentage DESC
"""

spark.sql(query_on_time).show()

+--------------------+------------------+
|                name|on_time_percentage|
+--------------------+------------------+
|Alaska Airlines Inc.| 72.82913165266106|
|Hawaiian Airlines...| 71.63742690058480|
|      Virgin America| 65.28477334366525|
|Delta Air Lines Inc.| 64.94491789648722|
|American Airlines...| 64.89963029728987|
|     US Airways Inc.| 60.78106739384496|
|United Air Lines ...| 60.61535839086338|
|SkyWest Airlines ...| 59.37500000000000|
|   Endeavor Air Inc.| 57.73022751895991|
|     JetBlue Airways| 55.71520087855770|
|Southwest Airline...| 54.90835030549898|
|           Envoy Air| 50.55119899988635|
|ExpressJet Airlin...| 49.14625366880180|
|  Mesa Airlines Inc.| 47.58735440931780|
|Frontier Airlines...| 42.18978102189781|
|AirTran Airways C...| 39.26380368098160|
+--------------------+------------------+



In [22]:
''' Find max distance flown for each airline '''

query_max_distance = """
SELECT 
    a.name,
    MAX(f.distance) AS max_distance
FROM flights_table f
JOIN airlines_table a 
    ON f.carrier = a.carrier
GROUP BY a.name
ORDER BY max_distance DESC
"""

spark.sql(query_max_distance).show()

+--------------------+------------+
|                name|max_distance|
+--------------------+------------+
|Hawaiian Airlines...|        4983|
|United Air Lines ...|        4963|
|      Virgin America|        2586|
|     JetBlue Airways|        2586|
|Delta Air Lines Inc.|        2586|
|American Airlines...|        2586|
|Alaska Airlines Inc.|        2402|
|     US Airways Inc.|        2153|
|Southwest Airline...|        2133|
|Frontier Airlines...|        1620|
|   Endeavor Air Inc.|        1587|
|ExpressJet Airlin...|        1389|
|           Envoy Air|        1147|
|SkyWest Airlines ...|        1008|
|AirTran Airways C...|         762|
|  Mesa Airlines Inc.|         544|
+--------------------+------------+

